In [3]:
#importing libraries
import sqlite3
import csv
import pandas as pd

In [4]:
con = sqlite3.connect('my_data1.db')
cur = con.cursor()

In [5]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv')
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method='multi')

999

In [6]:
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt


In [7]:
# sample creating a query, storing into a dataframe and displaying the dataframe
query = ('SELECT Orbit FROM SPACEXTBL') 
results = pd.read_sql_query(query, con)
results.head()

,Orbit
0,LEO
1,LEO (ISS)
2,LEO (ISS)
3,LEO (ISS)
4,LEO (ISS)


In [8]:
#Displaying the distinct launch sites
query = ('SELECT DISTINCT Launch_Site FROM SPACEXTBL')
results = pd.read_sql_query(query, con)
results.head()

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40
4,None


In [9]:
query = ('SELECT * FROM SPACEXTBL WHERE Launch_Site LIKE "CCA%"')
results = pd.read_sql_query(query, con)
results.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt


In [10]:
query = '''
         SELECT SUM(PAYLOAD_MASS__KG_) AS "total_NASA_mass" 
         FROM SPACEXTBL 
         WHERE Customer = "NASA (CRS)"
         '''
results = pd.read_sql_query(query, con)
results.head()

,total_NASA_mass
0,45596.0


In [11]:
query = '''
    SELECT AVG(PAYLOAD_MASS__KG_) AS "avg_mass_F9" 
    FROM SPACEXTBL 
    WHERE Booster_Version = "F9 v1.1" 
        '''
results = pd.read_sql_query(query, con)
results.head()

,avg_mass_F9
0,2928.4


In [12]:
query = '''
         SELECT MIN(Date) as "first_ground_pad" 
         FROM SPACEXTBL 
         WHERE Landing_Outcome = "Success (ground pad)"
         '''
results = pd.read_sql_query(query, con)
results.head()

,first_ground_pad
0,01/08/2018


In [13]:
query = '''
         SELECT Booster_Version, PAYLOAD_MASS__KG_ 
         FROM SPACEXTBL 
         WHERE Landing_Outcome = "Success (drone ship)" AND  PAYLOAD_MASS__KG_ BETWEEN 4000 and 6000
         '''
results = pd.read_sql_query(query, con)
results.head()

,Booster_Version,PAYLOAD_MASS__KG_
0,F9 FT B1022,4696.0
1,F9 FT B1026,4600.0
2,F9 FT B1021.2,5300.0
3,F9 FT B1031.2,5200.0


In [14]:
query = '''
        SELECT Mission_Outcome, COUNT(*)
        FROM SPACEXTBL
        WHERE Mission_Outcome IN ("Success", "Failure (in flight)")
        GROUP BY Mission_Outcome
        '''
results = pd.read_sql_query(query, con)
results.head()

,Mission_Outcome,COUNT(*)
0,Failure (in flight),1
1,Success,98


In [15]:
query = '''
        SELECT Booster_Version as "booster_version", PAYLOAD_MASS__KG_ as "payload"
        FROM SPACEXTBL 
        WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)
        '''
results = pd.read_sql_query(query, con)
results.head()

,booster_version,payload
0,F9 B5 B1048.4,15600.0
1,F9 B5 B1049.4,15600.0
2,F9 B5 B1051.3,15600.0
3,F9 B5 B1056.4,15600.0
4,F9 B5 B1048.5,15600.0


In [29]:
query = '''
        SELECT Landing_Outcome, Count(*) as Count
        FROM SPACEXTBL 
        WHERE substr(Date, 7, 4) || '-' || substr(Date, 4, 2) || '-' || substr(Date, 1, 2)
                BETWEEN "2010-06-04" AND "2017-03-20"
        GROUP BY Landing_Outcome
        ORDER BY Count DESC
        '''

In [30]:
results = pd.read_sql_query(query, con)
results.head()

,Landing_Outcome,Count
0,No attempt,10
1,Success (ground pad),5
2,Success (drone ship),5
3,Failure (drone ship),5
4,Controlled (ocean),3
